AttributeError: 'list' object has no attribute 'shape'

In [5]:
import numpy as np
from scipy.integrate import solve_ivp

# Define the kinetic rate constants
k1 = 6.0e-4   # kmol/m3/s
k2 = 4.0e-4   # kmol/m3/s

# Define the initial concentrations
NOx0 = 1.0e-3  # kmol/m3
NH30 = 0.0     # kmol/m3

# Define the thermal parameters
V = 0.002     # m^3
rho = 800.0   # kg/m^3
cp = 900.0    # J/kg.K
h = 50.0      # W/m^2.K
T0 = 300.0    # K
Tin = 800.0   # K
T_sur = 300.0 # K

# Define the function for the rate equations
def dCdt(t, C, T):
    NOx, NH3 = C
    dNOxdt = -k1*NOx*NH3
    dNH3dt = -dNOxdt/(2.0*k2)
    return [dNOxdt, dNH3dt]

# Define the function for the temperature distribution
def dTdt(t, T, C):
    Q = h*V*(Tin-T)
    dTdt = Q/(rho*cp*V) - k1*C[0]*C[1]/(rho*cp)
    return dTdt

# Define the time span for the simulation
tspan = [0.0, 100.0]   # seconds

# Define the initial conditions
C0 = [NOx0, NH30]
T0vec = np.ones_like(np.linspace(0, 1, 1))*T0

# Solve the rate equations and temperature distribution equations
sol = solve_ivp(lambda t, y: np.concatenate((dCdt(t, y[:2], y[2:]), dTdt(t, y[2:], y[:2]))),
                tspan, np.concatenate((C0, T0vec)), dense_output=True)

# Extract the concentrations and temperatures
t = np.linspace(0.0, 100.0, 1000)
C = sol.sol(t)[:2]
T = sol.sol(t)[2:]

# Calculate the conversion efficiency as a function of temperature
Tvec = np.linspace(300.0, 1000.0, 1000)
efficiency = np.zeros_like(Tvec)
for i in range(len(Tvec)):
    k1eff = k1*np.exp(-20000.0/(8.314*Tvec[i]))
    k2eff = k2*np.exp(-22000.0/(8.314*Tvec[i]))
    NOxsteady = NH30*(k2eff/k1eff)**0.5
    efficiency[i] = (NOx0 - NOxsteady)/NOx0
